In [ ]:
import glob
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import os
from unyt import uvstack
import yt
from yt.funcs import ensure_dir
from yt.utilities.physical_constants import G
from yt.visualization.color_maps import yt_colormaps

from yt.extensions.p2p.data_plotter import DataPlotter
from yt.extensions.p2p.model_profiles import \
    find_peaks, \
    load_model_profiles
from yt.extensions.p2p.stars import get_star_data

In [ ]:
pyplot.rcParams['figure.figsize'] = (15, 9)
pyplot.rcParams['font.size'] = 20

In [ ]:
yt.mylog.setLevel(40)

## Choose a star

In [ ]:
star_id = 334267081  ### first star
# star_id = 334267082  ### irradiated 1x, no heating, metal-free
# star_id = 334267083  ### irradiated 2x, heated 1x, metal-enriched (original target halo)
# star_id = 334267086  ### irradiated 2x, no heating, metal-free
# star_id = 334267090  ### irradiated 3x, heated 2x, metal-free
# star_id = 334267093  ### irradiated 4x, heated 2x, metal-free
# star_id = 334267099  ### irradiated 6x, minimal signs of heating, metal-free
# star_id = 334267102  ### irradiated 7x, heated 1x, metal-free
# star_id = 334267111  ### (target halo) irradiated 8x, multiple heatings, metal-enriched, some centering issues

In [ ]:
star_data = get_star_data("star_hosts.yaml")
my_star = star_data[star_id]
creation_time = my_star["creation_time"]

## Load all profiles

In [ ]:
profiles = load_model_profiles(star_id)

In [ ]:
ibefore = 0
for profile in profiles:
    if profile["None"].current_time < creation_time:
        ibefore += 1
iafter = len(profiles) - ibefore

## Plot a bunch of profiles

Change `x_field` below to one of the other values to plot profiles vs. enclosed gas or total mass.

In [ ]:
x_field = 'radius'
# x_field = 'm_gas_enc'
# x_field = 'm_tot_enc'

plots = (
    ('gas_density', x_field),
    ('dm_density', x_field),
    ('temperature', x_field),
    ('entropy', x_field),
    ('pressure', x_field),
    ('hyd_eq_pressure', x_field),
    ('H2_fraction', x_field),
    ('metallicity', x_field),
    ('m_gas_enc', 'radius'),
    ('baryon_fraction', x_field),
    ('mbe_ratio', x_field),
    ('timescale_ratio', x_field),
    ('turbulent_velocity', x_field),
    ('turbulent_mach_number', x_field),
)

labels = {
    'radius': 'r [pc]',
    'm_gas_enc': 'M$_{gas,enc}$ [Msun]',
    'm_tot_enc': 'M$_{tot,enc}$ [Msun]',
    'gas_density': '$\\rho_{b}\ [g/cm^{3}]$',
    'dm_density': '$\\rho_{dm}\ [g/cm^{3}]$',
    'temperature': 'T [K]',
    'entropy': 'entropy [erg cm$^{2}$]',
    'pressure': 'p [dyne/cm$^{2}$]',
    'hyd_eq_pressure': 'p / p$_{eq}$',
    'H2_fraction': 'f$_{H2}$',
    'metallicity': 'Z [Zsun]',
    'baryon_fraction': 'M$_{gas,enc}$ / M$_{tot,enc}$',
    'mbe_ratio': 'M$_{gas,enc}$ / M$_{BE}$',
    'timescale_ratio': 't$_{cs}$ / t$_{ff}$',
    'turbulent_velocity': 'v$_{turb}$ [km/s]',
    'turbulent_mach_number': '$M_{turb}$',
}

In [ ]:
### Bonnor-Ebert Mass constant
a = 1.67
b = (225 / (32 * np.sqrt(5 * np.pi))) * a**-1.5

n_plots = len(plots)
profile_data = []

plotter = DataPlotter(plots, labels)
pbar = yt.get_pbar("Plotting", len(profiles)-1)

for i, profile in enumerate(profiles):
    npds = profile["None"]
    vpds = profile["cell_volume"]
    mpds = profile["cell_mass"]
    pbar.update(i)

    before = npds.current_time < creation_time

    x_bins = npds.profile.x_bins
    used = npds.data['data', 'used'].d.astype(bool)

    m_tot = npds.profile['data', 'matter_mass'].to('Msun')[used]
    m_tot_enc = m_tot.cumsum()
    m_gas = npds.profile['data', 'cell_mass'].to('Msun')[used]
    m_gas_enc = m_gas.cumsum()
    m_dm = npds.profile['data', 'dark_matter_mass'].to('Msun')[used]
    m_dm_enc = m_dm.cumsum()

    r = npds.data['data', 'radius'][used].to('pc')

    rho_v = vpds.data['data', 'density'][used]
    rho_dm = vpds.data['data', 'dark_matter_density'][used]
    rho_total = vpds.data['data', 'matter_density'][used]

    plot_data = {
        'radius': r,
        'm_gas_enc': m_gas_enc,
        'm_tot_enc': m_tot_enc,
        'dm_density': rho_dm,
    }

    if mpds is not None:
        rho = mpds.data['data', 'density'][used]
        T = mpds.data['data', 'temperature'][used]
        H2 = mpds.data['data', 'H2_p0_fraction'][used]
        Z = mpds.data['data', 'metallicity3'][used].to('Zsun')
        p = mpds.data['data', 'pressure'][used]
        entropy = mpds.data['data', 'entropy'][used].to('erg*cm**2')
        
        cs = mpds.data['data', 'sound_speed'][used]
        v_turb = mpds.data['standard_deviation', 'velocity_magnitude'][used]
        cs_eff = np.sqrt(cs**2 + v_turb**2)

        m_BE = (b * (cs**4 / G**1.5) * p**-0.5).to('Msun')
        m_rat = m_gas_enc / m_BE

        # t_ff = mpds.data['data', 'total_dynamical_time'][used].to('Myr') / np.sqrt(2)
        t_ff = mpds.data['data', 'dynamical_time'][used].to('Myr') / np.sqrt(2)
        t_cool = mpds.data['data', 'cooling_time'][used].to('Myr')
        t_cs = (2 * r / cs_eff).to('Myr')
        t_rat = t_cs / t_ff
        # t_rat = t_cool / t_ff

        dr = np.diff(x_bins)[used]
        dp = (G * m_tot_enc * rho * dr / r**2)
        p_eq = dp[::-1].cumsum()[::-1]
        dp_dm = (G * m_dm_enc * rho * dr / r**2)
        p_eq_dm = dp_dm[::-1].cumsum()[::-1].in_cgs()

        plot_data.update({
            'gas_density': rho,
            'temperature': T,
            'entropy': entropy,
            'pressure': p,
            'hyd_eq_pressure': (p/p_eq_dm).to(""),
            'H2_fraction': H2,
            'metallicity': Z,
            'mbe_ratio': m_rat,
            'timescale_ratio': t_rat,
            'baryon_fraction': (m_gas_enc/m_tot_enc),
            'turbulent_velocity': v_turb.to("km/s"),
            'turbulent_mach_number': (v_turb/cs).to("")
        })

    if before:
        color = pyplot.cm.turbo(float(i/(ibefore-1)))
        alpha = 1
    else:
        color = pyplot.cm.winter(float(i-ibefore)/(iafter-1))
        alpha = 0.2
    plotter.plot_data(plot_data, color=color, alpha=alpha)

pbar.finish()

plotter.plot_axes()
plotter.axes['mbe_ratio'].set_ylim(1e-4, 3)
plotter.axes['mbe_ratio'].axhline(y=1, color='black', linestyle='--')
plotter.axes['timescale_ratio'].axhline(y=1, color='black', linestyle='--')
pyplot.show()

## Plot peaks in Bonnor-Ebert mass

In [ ]:
ds = yt.load(os.path.join("star_cubes", f"star_{star_id}_mass.h5"))

time_index = -1

rebin_field = ("data", "gas_mass_enclosed")
data_field = ("data", "bonnor_ebert_ratio")
i_peaks  = find_peaks(ds, rebin_field, data_field, time_index)

plot = True
if plot:
    bin_data = ds.data[rebin_field][time_index]
    peak_data = ds.data[data_field][time_index]
    peak_used = peak_data > 0

    pyplot.loglog(bin_data[peak_used], peak_data[peak_used])
    pyplot.scatter(bin_data[i_peaks], peak_data[i_peaks])
    pyplot.xlabel("M$_{gas, enc}$ [Msun]")
    pyplot.ylabel("peak field")
    pyplot.show()